# Lab 1 - Creating the SQL Tables

In this lab, use `sqlalchemy` to create, populate, and query a table from the baseball database, as well as for the `super_hero_powers.csv` table.  

In [1]:
import pandas as pd
artwork = pd.read_csv("./data/Artworks.csv")
import sqlalchemy
sqlalchemy.__version__

'1.2.7'

In [2]:
pd.__version__

'0.23.0'

## Part 1 - Baseball Managers

In this part of the lab, you will walk through the process of creating a manager table from [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/)

## Task 1 - Download, unzip, rename 

1. Download the baseball database linked above (save to desktop)
2. Unzip the file and rename to `baseball`
3. Load the `core/Managers.csv` file into a pandas `DataFrame` using `read_csv`
4. Inspect the `column` names and `dtypes`

In [3]:
import pandas as pd
managers = pd.read_csv('~/Desktop/baseball/core/Managers.csv')
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y


In [4]:
managers.columns

Index(['playerID', 'yearID', 'teamID', 'lgID', 'inseason', 'G', 'W', 'L',
       'rank', 'plyrMgr'],
      dtype='object')

**Question:** Is there a candidate for a primary key?

In [5]:
[(col, managers[col].is_unique) for col in managers]

[('playerID', False),
 ('yearID', False),
 ('teamID', False),
 ('lgID', False),
 ('inseason', False),
 ('G', False),
 ('W', False),
 ('L', False),
 ('rank', False),
 ('plyrMgr', False)]

**Solution:** Add the `index` as an actual column

In [6]:
from dfply import mutate
managers = (managers >>
            mutate(id = managers.index))

In [7]:
managers.id.is_unique

True

In [15]:
managers.columns

Index(['playerID', 'yearID', 'teamID', 'lgID', 'inseason', 'G', 'W', 'L',
       'rank', 'plyrMgr', 'id'],
      dtype='object')

In [8]:
managers.dtypes

playerID     object
yearID        int64
teamID       object
lgID         object
inseason      int64
G             int64
W             int64
L             int64
rank        float64
plyrMgr      object
id            int64
dtype: object

In [9]:
managers.shape

(3469, 11)

In [10]:
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr,id
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y,0
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y,1
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y,2
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y,3
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y,4


#### Task 2 - Create a `sqlalchemy` types `dict`

In [11]:
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID':String, 
             'plyrMgr':String,
             'teamID':String, 
             'lgID':String, 
             'yearID':Integer, 
             'inseason':Integer, 
             'G':Integer, 
             'W':Integer, 
             'L':Integer,
             'rank':Integer}

#### Task 4 - Create an `engine` and `schema`

In [11]:
!rm databases/baseball.db

rm: cannot remove 'databases/baseball.db': No such file or directory


In [12]:
from sqlalchemy import create_engine
mang_eng = create_engine("sqlite:///databases/baseball.db")
mang_eng.echo = True
schema = pd.io.sql.get_schema(managers, 'manager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)




#### Execute the `schema`

In [13]:
mang_eng.execute(schema)

2019-01-28 09:14:56,048 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-28 09:14:56,050 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 09:14:56,052 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-28 09:14:56,053 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 09:14:56,056 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)


2019-01-28 09:14:56,058 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 09:14:56,066 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
managers.shape

(3469, 11)

#### Task 5 - Use `to_sql` with `if_exists='append'` to insert the data

In [14]:
managers.to_sql('manager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-28 09:15:00,037 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("manager")
2019-01-28 09:15:00,038 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 09:15:00,046 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 09:15:00,769 INFO sqlalchemy.engine.base.Engine INSERT INTO manager ("playerID", "yearID", "teamID", "lgID", inseason, "G", "W", "L", rank, "plyrMgr", id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?,

2019-01-28 09:15:00,772 INFO sqlalchemy.engine.base.Engine ('wrighha01', 1871, 'BS1', None, 1, 31, 20, 10, 3.0, 'Y', 0, 'woodji01', 1871, 'CH1', None, 1, 28, 19, 9, 2.0, 'Y', 1, 'paborch01', 1871, 'CL1', None, 1, 29, 10, 19, 8.0, 'Y', 2, 'lennobi01', 1871, 'FW1', None, 1, 14, 5, 9, 8.0, 'Y', 3, 'deaneha01', 1871, 'FW1', None, 2, 5, 2, 3, 8.0, 'Y', 4, 'fergubo01', 1871, 'NY2', None, 1, 33, 16, 17, 5.0, 'Y', 5, 'mcbridi01', 1871, 'PH1', None, 1, 28, 21, 7, 1.0, 'Y', 6, 'hastisc01', 1871, 'RC1', None, 1, 25, 4, 21, 9.0, 'Y', 7, 'pikeli01', 1871, 'TRO', None, 1, 4, 1, 3, 6.0, 'Y', 8, 'cravebi01', 1871, 'TRO', None, 2, 25, 12, 12, 6.0, 'Y', 9, 'youngni99', 1871, 'WS3', None, 1, 32, 15, 15, 4.0, 'N', 10, 'cravebi01', 1872, 'BL1', None, 1, 41, 27, 13, 2.0, 'Y', 11, 'millsev01', 1872, 'BL1', None, 2, 17, 8, 6, 2.0, 'Y', 12, 'clintji01', 1872, 'BR1', None, 1, 11, 0, 11, 9.0, 'Y', 13, 'woodji01', 1872, 'BR1', None, 2, 18, 3, 15, 9.0, 'Y', 14, 'fergubo01', 1872, 'BR2', None, 1, 37, 9, 28, 7.0, 'Y

2019-01-28 09:15:00,837 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) too many SQL variables [SQL: 'INSERT INTO manager ("playerID", "yearID", "teamID", "lgID", inseason, "G", "W", "L", rank, "plyrMgr", id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?,

#### Task 6 - Query the table to make sure it all worked

In [14]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [15]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Manager = Base.classes.manager

In [16]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Manager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-28 08:59:23,120 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:59:23,122 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM manager
2019-01-28 08:59:23,123 INFO sqlalchemy.engine.base.Engine ()


[]

## Part 2 - Awards for Managers

Now add a table for the `AwardsManagers.csv` table.

In [18]:
Awards = pd.read_csv('~/Desktop/baseball/core/AwardsManagers.csv')
Awards.head()

,playerID,awardID,yearID,lgID,tie,notes
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN


In [19]:
Awards.columns

Index(['playerID', 'awardID', 'yearID', 'lgID', 'tie', 'notes'], dtype='object')

In [20]:
[(col, Awards[col].is_unique) for col in Awards]

[('playerID', False),
 ('awardID', False),
 ('yearID', False),
 ('lgID', False),
 ('tie', False),
 ('notes', False)]

In [21]:
Awards.shape

(179, 6)

In [22]:
Awards.head()

,playerID,awardID,yearID,lgID,tie,notes
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN


In [23]:
from dfply import mutate
Awards = (Awards >>
            mutate(id = Awards.index))

In [24]:
Awards.id.is_unique

True

In [25]:
Awards.dtypes

playerID    object
awardID     object
yearID       int64
lgID        object
tie         object
notes       object
id           int64
dtype: object

In [26]:
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID':String, 
             'awardID':String, 
             'lgID':String, 
             'yearID':Integer, 
             'LgID':String, 
             'tie':String, 
             'notes':String, 
             'id':Integer}

In [27]:
from sqlalchemy import create_engine
schema = pd.io.sql.get_schema(Awards, 'awards', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE awards (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT awards_pk PRIMARY KEY (id)
)




In [28]:
mang_eng.execute(schema)

2019-01-28 08:48:05,746 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE awards (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT awards_pk PRIMARY KEY (id)
)


2019-01-28 08:48:05,747 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:48:05,765 INFO sqlalchemy.engine.base.Engine COMMIT


## Part 3 - Super Hero Powers

Now make a database and table for the super hero powers.

## Problem 1
    
**Task:** One the `super_hero_powers.csv` and verify that the contents of the columns are all Boolean.  In this problem, you need to

1. Create a `dict` that defines the `pandas` column type
2. Read the file in using a `pd.read_csv`.
3. Clean up all the column labels.
    
**Be sure to write clean code!**


## Problem 2
    
Now define an `sqlalchemy` table for these data using `pandas` `to_sql` dataframe method.  You can use the `sqlalchemy.String` and `sqlalchemy.Boolean` columns type, which are [documented here](https://docs.sqlalchemy.org/en/latest/core/type_basics.html)

## Problem 3
    
Now you need to make a new `engine`, `inspect` your database, and make a `session` to query the database.

## Problem 4
    
Perform `sqlalchemy` queries to answer each of the following questions.

1. How many heroes have both Super Strength and Super Speed?
2. How many heroes have names that start with the word *Black*
3. Are heroes with Agility more likely to have Stealth?
4. What fraction of all heroes that can fly also have Super Strength?
5. Consider heroes that have names that contain `"girl"`, `"boy"`, `"woman"`, or `"man"`.  Compute the following ratio

$$\frac{N(\text{boy or man})}{N(\text{girl or woman}}$$

**Hint:** You will need to use some combination of `where`, `group_by`, and `count` for each part.

## Problem 5

Tell me another cool fact about the super powers.